# Experiments

Run this notebook to reproduce the experiments from the paper.

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler
# %load_ext cython

In [2]:
import math
import gudhi
from gudhi.hera import wasserstein_distance
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from miniball import Miniball
from scipy.spatial import KDTree

from datasets_morten import sample_circle, sample_rectangle, sample_torus, sample_cube, sample_flat_torus, sample_sphere
import warnings
import os
warnings.filterwarnings('ignore')

## Function definitions

### Compute distances to k-nearest neighbors in X

In [3]:
def k_nearest_neighbor_distances(X, k):
    kd_tree = KDTree(X)
    k_core_distances, _ = kd_tree.query(X, k=k, workers=-1)
    return k_core_distances  

### Compute filtration values of faces from squared radius and core values

In [4]:
def core_value(face, squared_radius, core_values):
    max_core = max(core_values[face]) 
    return max(squared_radius ** 0.5, max_core)

### Compute core value for each points

In [5]:
def vertex_values(X, max_k, max_r):
    if max_r is None or max_k <= 1:
        return k_nearest_neighbor_distances(X, k=[max_k])
    k_core_distances = k_nearest_neighbor_distances(X, np.arange(1, max_k + 1))
    line = np.linspace(max_r, 0, num=max_k)
    if max_k>1:
        indices = np.argmax(line <= k_core_distances, axis=1)
        values = k_core_distances[np.arange(len(k_core_distances)), indices]
        values[values > max_r] = max_r
    else:
        values = np.array(k_core_distances)
    return values

### Compute core simplex tree from a simplex tree filtered by squared radius and a point cloud

In [6]:
def core_complex(X, st, max_k, max_r=None):
    k_core_distances = vertex_values(X, max_k, max_r)**2
    for vertex in range(X.shape[0]):
        st.assign_filtration([vertex], k_core_distances[vertex])
    st.make_filtration_non_decreasing()
    return st

### Compute simplex tree of a point cloud filtered by squared radius

In [7]:
def cech_squared_radius(X, max_dim=1):
    st = gudhi.SimplexTree()
    for dim in range(max_dim+1):
        for face in combinations(range(len(X)),dim+1):
            val = Miniball(X[list(face)]).squared_radius()
            st.insert(face, val)
    return st

### Construct core cech simplex tree

In [8]:
def core_cech(X, max_k=10, max_r=None, max_dim=1):
    st = cech_squared_radius(X, max_dim=max_dim)
    return core_complex(X, st, max_k, max_r)

### Construct core alpha simplex tree

In [9]:
def core_alpha(X, max_k=10, max_r=None, precision='safe'):
    st = gudhi.AlphaComplex(points=X, precision=precision).create_simplex_tree()
    return core_complex(X, st, max_k, max_r)

In [10]:
def sqrt_persistence(st):
    persistence = st.persistence()
    return [(dim, (birth**.5, death**.5)) for dim, (birth, death) in persistence]

In [11]:
def plot_sqrt_persistence(st, axes=None):
    return gudhi.plot_persistence_diagram(sqrt_persistence(st), axes=axes)

In [12]:
def persistence_intervals_in_dimension(persistence, dim):
    return np.array([(b, d) for dimension, (b, d) in persistence if dimension == dim])    

In [13]:
def persistence_diagrams(
    X, max_ss: list[int] = [1, 10, 100, 1000], 
    max_r: float | None = -1
) -> list[tuple[int, tuple[int, int]]]:
    if max_r < 0:
        max_r = 2*math.sqrt(Miniball(X).squared_radius())
    res = []
    for i, max_s in enumerate(max_ss):
        max_k = max(1, int((M + N) * max_s))
        st = core_alpha(X, max_k=max_k, max_r=max_r)
        persistence = sqrt_persistence(st)
        res.append(persistence)
    return res

In [14]:
def bottleneck_distances(pers1, pers2, dim):
    assert len(pers1) == len(pers2)
    n = len(pers1)
    A = np.zeros(n)
    for i in range(n):
        a = persistence_intervals_in_dimension(pers1[i], dim)
        b = persistence_intervals_in_dimension(pers2[i], dim)
        bdist = gudhi.bottleneck_distance(a, b)
        A[i] = bdist
    return A

In [15]:
def plot_persistence_diagrams(pers_diagrams, max_ks, max_r, title="Alpha Čech"):
    fig, axs = plt.subplots(ncols=len(max_ks), figsize=(14, 14 / len(max_ks)))
    fig.suptitle(title)
    for i, (pers, max_k) in enumerate(zip(pers_diagrams, max_ks)):
        gudhi.plot_persistence_diagram(pers, axes=axs[i])
        # st_core = core_cech(X, max_dim=2, max_k=max_k, max_r=max_r)
        # plot_sqrt_persistence(st_core, axes=axs[i])
        # gudhi.plot_persistence_diagram(st_core.persistence(), axes=axs[i])
        axs[i].set_title(f"max_k={max_k}, max_r={max_r:.2f}")
    fig.tight_layout()
    return fig    

In [16]:
def circle_with_noise(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_circle(N, rng, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]    

In [17]:
def two_circles_with_noise(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    N1 = (2 * N) // 3
    N2 = N // 3 
    
    Z1 = sample_circle(N1, rng, r=1, std=sigma)
    Z2 = sample_circle(N2, rng, r=0.5, std=sigma)
    Z = np.r_[Z1, Z2]
    
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z1, Z2, Y]

In [18]:
def embedded_torus(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_torus(N, rng, a=1, b=3, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]

In [19]:
def sphere(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_sphere(N, rng, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]

In [20]:
def clifford_torus(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_flat_torus(N, rng, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]

In [21]:
def bottleneck_distance_experiment(
    Ms, Ns, max_ss=[0, 0.001, 0.01, 0.1], sigma=0.07, max_r=-1, 
    point_generator=circle_with_noise, seed=0):
    rng = np.random.default_rng(seed=seed)
    X = point_generator(0, 5, sigma=0, rng=rng)
    dimensions = range(X.shape[1])
    res = {dim: [] for dim in dimensions}
    for M, N in zip(Ms, Ns):
        print(f"M={M}, N={N}")
        X = point_generator(0, M + N, sigma=0, rng=rng)
        st_ideal = core_alpha(X, max_k=1, max_r=None)#max_r)
        persistence_ideal = sqrt_persistence(st_ideal)    
        for max_s in max_ss:
            X = point_generator(M, N, sigma, rng=rng)
            if max_r is not None:
                max_r = 2*math.sqrt(Miniball(X).squared_radius())
            max_k = max(1, int((M + N) * max_s))
            print(f"\tmax_s={max_s} (max_k = {max_k})")
            st = core_alpha(X, max_k=max_k, max_r=max_r)
            persistence = sqrt_persistence(st)
            for dim in dimensions:
                res[dim].append(
                    wasserstein_distance(
                        persistence_intervals_in_dimension(persistence, dim),
                        persistence_intervals_in_dimension(persistence_ideal, dim), order=1, internal_p=1))
    return res
        
        

In [22]:
def formatted_bottleneck_results(bottleneck_distances, Ms, Ns, max_ss):
    res = [f"Ns={Ns} Ms={Ms} max_ss={max_ss}"]
    for dim in bottleneck_distances.keys():
        res.append(f"Dim {dim} & " + " & ".join([f"{dist:.3f}" for dist in bottleneck_distances[dim]]))
    return res

In [23]:
def run_experiments(
    point_generators,
    Ms,
    Ns,
    max_ss,
    names=None,
    sigma=0.07,
    seed=0,
    max_r=-1,
):
    res = []
    for idx, generator in enumerate(point_generators):
        if names is not None:
            name = names[idx]
        else:
            name = 'Unknown'
        print(f"Running experiments for {name}")
        bottleneck_dists = bottleneck_distance_experiment(
            Ms = Ms, Ns=Ns, max_ss=max_ss, point_generator=generator, sigma=sigma, seed=seed, max_r=max_r)
        res.append([name] +
                   formatted_bottleneck_results(bottleneck_dists, Ms, Ns, max_ss))
    return res

In [24]:
Ms = [10_000, 1_000, 100]
Ns = [10_000, 10_000, 10_000]
max_ss=[0, 0.001, 0.01, 0.1]
sigma = 0.07
seed = 0
point_generators = [circle_with_noise, two_circles_with_noise, sphere, embedded_torus, clifford_torus]
names = ["Circle", "Circles", "Sphere", "Torus 1", "Torus 2"]

# Persistence along a line

In [ ]:
%%time
result = run_experiments(
    point_generators,
    Ms = Ms,
    Ns = Ns,
    max_ss = max_ss,
    sigma=sigma,
    seed=seed,
    names = names)

Running experiments for Circle
M=10000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 20)
	max_s=0.01 (max_k = 200)
	max_s=0.1 (max_k = 2000)
M=1000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 11)
	max_s=0.01 (max_k = 110)
	max_s=0.1 (max_k = 1100)
M=100, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 10)
	max_s=0.01 (max_k = 101)
	max_s=0.1 (max_k = 1010)
Running experiments for Circles
M=10000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 20)
	max_s=0.01 (max_k = 200)
	max_s=0.1 (max_k = 2000)
M=1000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 11)
	max_s=0.01 (max_k = 110)
	max_s=0.1 (max_k = 1100)
M=100, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 10)
	max_s=0.01 (max_k = 101)
	max_s=0.1 (max_k = 1010)
Running experiments for Sphere
M=10000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 20)
	max_s=0.01 (max_k = 200)
	max_s=0.1 (max_k = 2000)
M=1000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 11)
	max_s=0.01 (max_k = 110)
	max_s=0.1 

In [ ]:
print('\n'.join(['\n'.join(x) for x in result]))

# Persistence for fixed $s$ (and $k$)

In [ ]:
%%time
result_fixed_k = run_experiments(
    point_generators,
    Ms = Ms,
    Ns = Ns,
    max_ss = max_ss,
    sigma=sigma,
    seed=seed,
    names = names,
    max_r=None,
)

In [ ]:
print('\n'.join(['\n'.join(x) for x in result_fixed_k]))

# Runtime analysis

In [46]:
import timeit
import functools

In [48]:
repeat = 10
Ns = [10000, 20000, 30000, 40000, 50000, 60000]
ks = [10, 100, 1000, 10000]
sigma = 0.07

def alpha_core_persistence(X, k):
    st = core_alpha(X, max_k=k, max_r=None)
    st.persistence()

print(f"Ns = {Ns}")
for k in ks:
    print(f"\nk = {k}")
    for N in Ns:
        X = embedded_torus(N // 2, N // 2, sigma, rng=None, seed=0)
        times = timeit.repeat(functools.partial(alpha_core_persistence, X, k), number=1, repeat=repeat)
        print(f"Size {N}: {min(times):.2f}s.")

Ns = [10000, 20000, 30000, 40000, 50000, 60000]

k = 1
Size 10000: 1.18s.
Size 20000: 2.63s.
Size 30000: 4.21s.
Size 40000: 5.80s.
Size 50000: 7.38s.
Size 60000: 8.73s.

k = 10
Size 10000: 1.11s.
Size 20000: 2.32s.
Size 30000: 3.57s.
Size 40000: 4.85s.
Size 50000: 6.17s.
Size 60000: 7.51s.

k = 100
Size 10000: 1.14s.
Size 20000: 2.42s.
Size 30000: 3.75s.
Size 40000: 5.08s.
Size 50000: 6.53s.
Size 60000: 7.91s.

k = 1000
Size 10000: 1.72s.
Size 20000: 3.59s.
Size 30000: 5.58s.
Size 40000: 7.50s.
Size 50000: 9.76s.
Size 60000: 11.70s.

k = 10000
Size 10000: 7.86s.
Size 20000: 16.90s.
Size 30000: 26.71s.
Size 40000: 35.15s.
Size 50000: 47.29s.
Size 60000: 56.64s.
